## Girvan–Newman — The Clustering Technique in Network Analysis

Girvan-Newman method is one of the classic community clustering techniques, which separates the network based on the betweenness of the edges.. By using the algorithm, we are able to separate the network into communities, and the community detection can be used as a good start of data preprocessing.

In this example we will implement the Girvan-Newman clustering algorithm. Please refer to my previous post to see the details of the algorithm.

previous post: https://medium.com/analytics-vidhya/girvan-newman-the-clustering-technique-in-network-analysis-27fe6d665c92



In [2]:
import numpy as np
import pandas as pd
import itertools 
import math
import time
from collections import defaultdict
from copy import deepcopy


First we will define a simple graph as following.


In [3]:
# edge_dict is the dictionary of edge connection, such that 
# edge_dict[source] = [output_vertices]

edge_dict = defaultdict(lambda: [])
edge_dict['a'].extend(['b','c','d'])
edge_dict['b'].extend(['a','c'])
edge_dict['c'].extend(['a','b','d'])
edge_dict['d'].extend(['a','c','e'])
edge_dict['e'].extend(['d','f','g','h'])
edge_dict['f'].extend(['e','g'])
edge_dict['g'].extend(['e','f','h'])
edge_dict['h'].extend(['e','g'])

edge_dict

defaultdict(<function __main__.<lambda>()>,
            {'a': ['b', 'c', 'd'],
             'b': ['a', 'c'],
             'c': ['a', 'b', 'd'],
             'd': ['a', 'c', 'e'],
             'e': ['d', 'f', 'g', 'h'],
             'f': ['e', 'g'],
             'g': ['e', 'f', 'h'],
             'h': ['e', 'g']})

### Calculate the betweenness value of the current graph

We use the following formula to calculate the edge credit

$$EdgeCredit= (1+\sum{IncomingEdgeCredit}) * \frac{ScoreOfDestination} {ScoreOfStart} $$

By traversing all the vertices and we can get the edge betweenness by summing all the edge credit.

In [5]:
def scale_DAG_btw_dict(edge_btw_dict):
    for e in edge_btw_dict:
        edge_btw_dict[e] /= 2
    return edge_btw_dict

def calculate_btw_and_communities(edge_dict, vertices):
    edge_btw_dict = defaultdict(lambda:0)
    com_res = set()
    
    # first we select a node v to start, and perform bfs to assign node score and edge credits
    # where node_score contains {node: [node_score, edge_credit]}
    
    for v in vertices:
        visited = []
        src = [v]
        node_score = defaultdict(lambda:[0,1])
        node_score[v][0] += 1
        edge_path = []

        while True:
            visited.extend(src)
            next_src = set()
            cur_level_edge = defaultdict(lambda:[])
            for v in src:
                edge = edge_dict[v]
                for next_node in edge:
                    if next_node not in visited:
                        next_src.add(next_node)
                        node_score[next_node][0]+=node_score[v][0]
                        cur_level_edge[next_node].append(v)


            if len(next_src) == 0:
                if v not in edge_dict:
                    com_res.add(tuple([v]))
                else:
                    com_res.add(tuple(sorted(visited)))
                break
            else:
                edge_path.append(cur_level_edge)
                src = next_src
        
        # next, we compute the betweenness value and the edge credit to the next node
        for indx in range(len(edge_path)-1, -1, -1):
            level_nodes = edge_path[indx]

            for v in level_nodes:
                edges = level_nodes[v]
                for next_node in edges:
                    btw_val = node_score[v][1] * node_score[next_node][0] / node_score[v][0]

                    edge_btw_dict[tuple(sorted([v,next_node]))] += btw_val
                    node_score[next_node][1] += btw_val
    
    edge_btw_dict = scale_DAG_btw_dict(edge_btw_dict)
    
    return edge_btw_dict, com_res

edge_btw_dict, communities = calculate_btw_and_communities(edge_dict,list(edge_dict.keys()))
edge_btw_dict

defaultdict(<function __main__.calculate_btw_and_communities.<locals>.<lambda>()>,
            {('e', 'f'): 5.5,
             ('e', 'g'): 5.0,
             ('e', 'h'): 5.5,
             ('d', 'e'): 16.0,
             ('a', 'b'): 3.5,
             ('a', 'c'): 1.0,
             ('a', 'd'): 7.5,
             ('c', 'd'): 7.5,
             ('b', 'c'): 3.5,
             ('g', 'h'): 1.5,
             ('f', 'g'): 1.5})

### Compute the best community

We remove the edges with highest betweenness value iteratively and aim to optimize the modularity, such that 

$$
Q(G,S) = \frac{1}{2m} \sum_{s\in S}\sum_{i\in s}\sum_{j\in s} (A_{ij} - \frac{k_ik_j}{2m})
$$


In [6]:
def compute_modularity(original_graph, update_graph, m, communities):
    res = 0
    for com in communities:
        for edge in itertools.combinations(com,2):
            A = 0
            v1,v2 = edge
            if v1 in original_graph[v2] and v2 in original_graph[v1]:
                A = 1
            ki = len(update_graph[v1])
            kj = len(update_graph[v2])
            res += A - (ki*kj)/(2*m)
    return res/(2*m)

def remove_highest_btw_edge(edge_dict, edge_btw_dict):
    largest_key = sorted(edge_btw_dict, key = lambda x: edge_btw_dict[x], reverse = True)[0]
    remove_key_list = list(filter(lambda x: round(edge_btw_dict[x],5) == round(edge_btw_dict[largest_key],5), edge_btw_dict))
    
    print(f"remove edges {remove_key_list} with betweenness value around {edge_btw_dict[largest_key]}")
    for e in remove_key_list:
        
        edge_dict[e[0]].remove(e[1])
        edge_dict[e[1]].remove(e[0])
    return edge_dict
    
def compute_opt_communities(edge_dict, vertices, verbose = True):
    max_modularity = float('-inf')
    edge_set = set()

    for k in edge_dict:
        edge_set.update([tuple(sorted([k,j])) for j in edge_dict[k]])
    edge_count = len(edge_set)
    update_graph = deepcopy(edge_dict)
    edge_btw_dict, current_best_community = calculate_btw_and_communities(edge_dict, vertices)
    
    while True:
        update_graph = remove_highest_btw_edge(update_graph, edge_btw_dict)
        edge_btw_dict, next_community = calculate_btw_and_communities(update_graph, vertices)
        
        cur_modularity = compute_modularity(edge_dict, update_graph, edge_count, next_community)
        if cur_modularity >= max_modularity:
            if verbose:
                print(f"update best modularity of community split {max_modularity} ---> {cur_modularity}\n")
            max_modularity = cur_modularity
            current_best_community = next_community
        else:
            if verbose:
                print(f"modularity after split = {cur_modularity}, which is lower than best split {max_modularity}")
            break
    return current_best_community

best_community = compute_opt_communities(edge_dict, list(edge_dict.keys()))
best_community

remove edges [('d', 'e')] with betweenness value around 16.0
update best modularity of community split -inf ---> 0.3016528925619835

remove edges [('a', 'b'), ('a', 'd'), ('c', 'd'), ('b', 'c'), ('e', 'f'), ('e', 'h'), ('g', 'h'), ('f', 'g')] with betweenness value around 1.5
modularity after split = 0.08677685950413223, which is lower than best split 0.3016528925619835


{('a', 'b', 'c', 'd'), ('e', 'f', 'g', 'h')}

As we can see from the result, we get the best community split by removing the edge (d,e)

### A more complex network

Next, we will generate a more complex network 

In [7]:
edge_dict = defaultdict(lambda: [])
edge_dict['a'].extend(['b','c','d','p'])
edge_dict['b'].extend(['a','c'])
edge_dict['c'].extend(['a','b','d'])
edge_dict['d'].extend(['a','c','e'])
edge_dict['e'].extend(['d','f','g','h'])
edge_dict['f'].extend(['e','g'])
edge_dict['g'].extend(['e','f','h'])
edge_dict['h'].extend(['e','g','j'])
edge_dict['i'].extend(['j','l'])
edge_dict['j'].extend(['h','i','l','k'])
edge_dict['k'].extend(['j','l'])
edge_dict['l'].extend(['i','j','k'])
edge_dict['m'].extend(['n','o','p'])
edge_dict['n'].extend(['m','o'])
edge_dict['o'].extend(['m','n','p'])
edge_dict['p'].extend(['a','m','o'])


In [8]:
best_community = compute_opt_communities(edge_dict, list(edge_dict.keys()))
best_community

remove edges [('d', 'e')] with betweenness value around 64.0
update best modularity of community split -inf ---> 0.2797731568998111

remove edges [('a', 'p'), ('h', 'j')] with betweenness value around 16.0
update best modularity of community split 0.2797731568998111 ---> 0.3648393194706995

remove edges [('a', 'b'), ('a', 'd'), ('c', 'd'), ('b', 'c'), ('e', 'f'), ('e', 'h'), ('g', 'h'), ('f', 'g'), ('j', 'k'), ('k', 'l'), ('i', 'j'), ('i', 'l'), ('m', 'n'), ('m', 'p'), ('o', 'p'), ('n', 'o')] with betweenness value around 1.5
modularity after split = 0.08506616257088848, which is lower than best split 0.3648393194706995


{('a', 'b', 'c', 'd'),
 ('e', 'f', 'g', 'h'),
 ('i', 'j', 'k', 'l'),
 ('m', 'n', 'o', 'p')}